In [1]:
import pandas as pd
import matplotlib as plt
import datetime
import time
import numpy as np
import gc

In [2]:
#pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
#pd.set_option('display.width', 1000)

# Market Data

In [ ]:
df1 = pd.read_csv("train_market/market_train1.csv")
df2 = pd.read_csv("train_market/market_train2.csv")
df3 = pd.read_csv("train_market/market_train3.csv")
df4 = pd.read_csv("train_market/market_train4.csv")
df5 = pd.read_csv("train_market/market_train5.csv")
market_data = pd.concat([df1,df2,df3,df4,df5])

In [ ]:
market_data = market_data.drop(market_data.columns[0], axis=1)
market_data['time'] = pd.to_datetime(market_data['time'])
market_data['assetName'] = market_data['assetName'].astype('category')
market_data.sort_values(['assetCode', 'time'], inplace=True)
market_data['returnsOpenNextMktres1'] = market_data.groupby('assetCode')['returnsOpenNextMktres10'].shift(9)
market_data['returnsOpenNextMktres2'] = market_data.groupby('assetCode')['returnsOpenNextMktres10'].shift(8)
market_data['returnsOpenNextMktres4'] = market_data.groupby('assetCode')['returnsOpenNextMktres10'].shift(6)
market_data['returnsOpenNextMktres6'] = market_data.groupby('assetCode')['returnsOpenNextMktres10'].shift(4)

print("Min timestamp : {}, Max timestamp : {}, Market data shape : {}".format(market_data['time'].min(), 
                                                                              market_data['time'].max(),
                                                                              market_data.shape))
market_data = market_data[market_data['time'].dt.year >= 2009]
print("Shape of market data after time filtering : {}".format(market_data.shape))

In [ ]:
market_data.head()

In [ ]:
market_data.columns

In [ ]:
columns = ['time', 'assetName', 'volume', 'close', 'open', 'returnsOpenNextMktres10'] 

In [ ]:
market_data = market_data[columns]

In [ ]:
market_data.shape

In [ ]:
market_data['assetName'].unique()

In [ ]:
agg_data = market_data.groupby(['assetName']).count()['time'].reset_index(name='count').sort_values(['count'], ascending=False)

In [ ]:
agg_data['assetName'].iloc[1]

In [ ]:
agg_data.shape

# News data

In [3]:
#Using 2010 to 2016 data
def process_news_data(news_data):
    print("Shape : {}".format(news_data.shape))
    news_data = news_data.drop(news_data.columns[0], axis=1)
    news_data['time'] = pd.to_datetime(news_data['time'])
    news_data['sourceTimestamp'] = pd.to_datetime(news_data['sourceTimestamp'])
    news_data['firstCreated'] = pd.to_datetime(news_data['firstCreated'])
    news_data['provider'] = news_data['provider'].astype('category')
    news_data['subjects'] = news_data['subjects'].astype('category')
    news_data['audiences'] = news_data['audiences'].astype('category')
    news_data['assetCodes'] = news_data['assetCodes'].astype('category')
    news_data['assetName'] = news_data['assetName'].astype('category')
    news_data = news_data[news_data['time'].dt.year >= 2009]
    
    news_data['rel_firstMention'] = 1.0*news_data['firstMentionSentence']/news_data['sentenceCount']
    news_data['rel_SentCount'] = 1.0*news_data['sentimentWordCount']/news_data['wordCount']
    
    return news_data

In [4]:
df1 = pd.read_csv("train_news/news_train1.csv")
df1 = process_news_data(df1)
df2 = pd.read_csv("train_news/news_train2.csv")
df2 = process_news_data(df2)
df3 = pd.read_csv("train_news/news_train3.csv")
df3 = process_news_data(df3)
df4 = pd.read_csv("train_news/news_train4.csv")
df4 = process_news_data(df4)
df5 = pd.read_csv("train_news/news_train5.csv")
df5 = process_news_data(df5)
df6 = pd.read_csv("train_news/news_train6.csv")
df6 = process_news_data(df6)
df7 = pd.read_csv("train_news/news_train7.csv")
df7 = process_news_data(df7)
df8 = pd.read_csv("train_news/news_train8.csv")
df8 = process_news_data(df8)
df9 = pd.read_csv("train_news/news_train9.csv")
df9 = process_news_data(df9)
df10 = pd.read_csv("train_news/news_train10.csv")
df10 = process_news_data(df10)
news_data = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10])

Shape : (1000000, 36)
Shape : (1000000, 36)
Shape : (1000000, 36)
Shape : (1000000, 36)
Shape : (1000000, 36)
Shape : (1000000, 36)
Shape : (1000000, 36)
Shape : (1000000, 36)
Shape : (1000000, 36)
Shape : (328749, 36)


In [5]:
news_data = news_data[pd.notnull(news_data['headline'])]
news_data.shape

(7770025, 37)

In [6]:
## Removing late report of news
news_data['news_delay'] = news_data['time'] - news_data['sourceTimestamp']
news_data = news_data[news_data.news_delay < datetime.timedelta(days=1)]
news_data['time'] = news_data['time'].dt.date
news_data.shape

(7770021, 38)

In [7]:
news_data1 = news_data[pd.notnull(news_data['assetName'])]
news_data1.shape

(7770021, 38)

In [8]:
news_data.columns

Index(['time', 'sourceTimestamp', 'firstCreated', 'sourceId', 'headline',
       'urgency', 'takeSequence', 'provider', 'subjects', 'audiences',
       'bodySize', 'companyCount', 'headlineTag', 'marketCommentary',
       'sentenceCount', 'wordCount', 'assetCodes', 'assetName',
       'firstMentionSentence', 'relevance', 'sentimentClass',
       'sentimentNegative', 'sentimentNeutral', 'sentimentPositive',
       'sentimentWordCount', 'noveltyCount12H', 'noveltyCount24H',
       'noveltyCount3D', 'noveltyCount5D', 'noveltyCount7D', 'volumeCounts12H',
       'volumeCounts24H', 'volumeCounts3D', 'volumeCounts5D', 'volumeCounts7D',
       'rel_firstMention', 'rel_SentCount', 'news_delay'],
      dtype='object')

In [9]:
news_col = ['time', 'headline', 'urgency', 'sentenceCount', 'wordCount', 'assetName', 'firstMentionSentence', 
           'relevance', 'sentimentClass','sentimentWordCount','rel_firstMention', 'rel_SentCount']

In [10]:
news_data1 = news_data1[news_col]

In [11]:
gc.collect()

0

In [26]:
conn = sqlite3.connect("NewsData.db")

In [28]:
cnt = 0
for i in asset_list:
    print(i)
    if i != 'Unknown':
        table_name = re.sub(r'\W+', '', i)
        table_name = "News_" + table_name
        #print(table_name)
        #ind_stock = market_data[market_data['assetName'] == i]
        ind_stock = news_data1[news_data1['assetName'] == i]
        ind_stock = ind_stock.sort_values( 'time')
        ind_stock.to_sql(table_name, conn)

3M Co
A. O. Smith Corp
ABB Ltd
ADTRAN Inc
AECOM
AES Corp
AGCO Corp
ANSYS Inc
ARM Holdings PLC
ASE Technology Holding Co Ltd
AT&T Inc
AU Optronics Corp
AXIS Capital Holdings Ltd
Abbott Laboratories
Abercrombie & Fitch Co
Acadia Realty Trust
Acas LLC
Accenture PLC
Acorda Therapeutics Inc
Activision Blizzard Inc
Actuant Corp
Acxiom Corp
Adobe Systems Inc
Advance Auto Parts Inc
Advanced Energy Industries Inc
Advanced Micro Devices Inc
Aegon NV
AerCap Holdings NV
Aetna Inc
Affiliated Managers Group Inc
Aflac Inc
Agilent Technologies Inc
Agnico Eagle Mines Ltd
Agrium Inc
Air Products and Chemicals Inc
Aircastle Ltd
Airgas Inc
Akamai Technologies Inc
Alaska Air Group Inc
Albemarle Corp
Alcatel Lucent SA
Alexandria Real Estate Equities Inc
Alexion Pharmaceuticals Inc
Align Technology Inc
Allegheny Technologies Inc
Allergan plc
Alliance Data Systems Corp
Alliant Energy Corp
Allscripts Healthcare Solutions Inc
Allstate Corp
Altaba Inc
Altair Engineering Inc
Altria Group Inc
Amazon.com Inc
Amdocs

Gold Fields Ltd
Goldcorp Inc
Goldman Sachs Group Inc
Goodyear Tire & Rubber Co
Grand Canyon Education Inc
Graphic Packaging Holding Co
Great Plains Energy Inc
Grupo Televisa SAB
H & R Block Inc
HCP Inc
HDFC Bank Ltd
HSBC Holdings PLC
HSN Inc
Hain Celestial Group Inc
Halliburton Co
Halozyme Therapeutics Inc
HanesBrands Inc
Harley-Davidson Inc
Harman International Industries Inc
Harmony Gold Mining Company Ltd
Harris Corp
Harsco Corp
Hartford Financial Services Group Inc
Hasbro Inc
Hatteras Financial Corp
Hawaiian Electric Industries Inc
Health Net Inc
HealthSouth Corp
Healthcare Realty Trust Inc
Heartland Express Inc
Hecla Mining Co
Helix Energy Solutions Group Inc
Helmerich and Payne Inc
Henry Schein Inc
Herbalife Nutrition Ltd
Herman Miller Inc
Hershey Co
Hertz Global Holdings Inc
Hess Corp
Hexcel Corp
Hibbett Sports Inc
Highwoods Properties Inc
Hill-Rom Holdings Inc
Hologic Inc
Home Depot Inc
Honda Motor Co Ltd
Honeywell International Inc
Hormel Foods Corp
Hospitality Properties Trus

Siliconware Precision Industries Co Ltd
Simon Property Group Inc
Skechers USA Inc
Skyworks Solutions Inc
Sociedad Quimica y Minera de Chile SA
Sohu.com Ltd
Sonic Corp
Sonoco Products Co
Sony Corp
Sotheby's
Southern Co
Southern Company Gas
Southern Copper Corp
Southwest Airlines Co
Southwestern Energy Co
Spirit AeroSystems Holdings Inc
St Jude Medical Inc
St. Joe Co
Stanley Black & Decker Inc
Staples Inc
Starbucks Corp
Starwood Hotels & Resorts Worldwide Inc
Starwood Property Trust Inc
State Street Corp
Steel Dynamics Inc
Steelcase Inc
Stericycle Inc
Steven Madden Ltd
Stifel Financial Corp
Stillwater Mining Co
Stryker Corp
Sun Life Financial Inc
SunPower Corp
SunTrust Banks Inc
Suncor Energy Inc
Sunstone Hotel Investors Inc
Superior Energy Services Inc
Symantec Corp
Synaptics Inc
Synopsys Inc
Sysco Corp
T-Mobile US Inc
T. Rowe Price Group Inc
TCF Financial Corp
TD Ameritrade Holding Corp
TE Connectivity Ltd
TECO Energy Inc
TFS Financial Corp
TJX Companies Inc
Taiwan Semiconductor Manufa

In [30]:
conn.close()

In [15]:
with open("asset.json", "r") as f:
    content = f.read()
asset_list = [i.replace('"','') for i in content.split("\n")]

In [23]:
asset_list = asset_list[:-1]

In [27]:
len(asset_list)

1000

In [ ]:
cnt = 0
for i in agg_data['assetName']:
    if cnt >=1000:
        break
    print(i)
    if i != 'Unknown' and '&' not in i:
        print(i)
        table_name = re.sub(r'\W+', '', i)
        table_name="Market_" + table_name
        #print(table_name)
        #ind_stock = market_data[market_data['assetName'] == i]
        ind_stock = market_data[market_data['assetName'] == i]
        #ind_stock = ind_stock.sort_values( 'time').head()
        ind_stock.to_sql(table_name, conn)
        asset_list.append(i)
        cnt += 1

# Sqlite Experimentation

In [25]:
import sqlite3
from pandas.io import sql
import re

In [ ]:
conn = sqlite3.connect("MarketData.db")

In [ ]:
#cur = conn.cursor()
#market_data[market_data['assetName'] == "Morgan Stanley"]
asset_list = []

In [ ]:
cnt = 0
for i in agg_data['assetName']:
    if cnt >=1000:
        break
    print(i)
    if i != 'Unknown' and '&' not in i:
        print(i)
        table_name = re.sub(r'\W+', '', i)
        table_name="Market_" + table_name
        #print(table_name)
        #ind_stock = market_data[market_data['assetName'] == i]
        ind_stock = market_data[market_data['assetName'] == i]
        #ind_stock = ind_stock.sort_values( 'time').head()
        ind_stock.to_sql(table_name, conn)
        asset_list.append(i)
        cnt += 1

In [ ]:
conn.close()

In [ ]:
asset_list = []

In [ ]:
with open("asset.json", "r") as f:
    content = f.read()
    print(content)
content.split("\n")[0].replace('"','')

In [ ]:
content

In [ ]:
if '&' in x:
    print("ADITYA")

In [ ]:
agg_data['assetName'].reset_index().drop(news_data.columns[0], axis=1)

In [ ]:
ag_data1 = agg_data['assetName'].reset_index()

In [ ]:
ag_data1 = ag_data1[1:].drop(ag_data1.columns[0], axis=1)

In [ ]:
with open('asset.json', 'w') as f:
    f.write(ag_data2.to_json(orient='records', lines=True))

In [ ]:
ag_data2 = ag_data1.rename(columns = {'assetName':'label'}).sort_ca

In [ ]:
ag_data2.sort_values('label', inplace=True)

In [ ]:
table_name

In [ ]:
i

In [ ]:
ind_stock = market_data[market_data['assetName'] == i]
ind_stock = ind_stock.sort_values( 'time').head()

In [ ]:
type(ind_stock)

In [ ]:
table_name = re.sub(r'\W+', '', asset_name)

In [ ]:
ind_stock.columns

In [ ]:
ind_stock.sort_values( 'time').head()

# Rough Work

In [ ]:
import re
re.sub(r'\W+', '', your_string)